In [1]:
import base64
import datetime
import time
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from numpy import median
import matplotlib.dates as mdates

np.random.seed(1337)
%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

## Load the Data

In [2]:
tr = pd.read_csv('input/train_ver2.csv',
        dtype={
        'age': np.str,
        'antiguedad': np.str,
        'indrel_1mes': np.str,
        'conyuemp': np.str},
        parse_dates=[
        'fecha_dato',
        'fecha_alta'])
len(tr)

13647309

In [3]:
te = pd.read_csv('input/test_ver2.csv',
        dtype={
        'age': np.str,
        'antiguedad': np.str,
        'indrel_1mes': np.str,
        'conyuemp': np.str},
        parse_dates=[
        'fecha_dato',
        'fecha_alta'])
len(te)

929615

In [4]:
for c in tr.columns[24:]:
    te[c] = np.nan

In [5]:
d = pd.concat([tr, te])
len(d)

14576924

In [6]:
del tr
del te

## measurement_date

In [7]:
d.rename(columns={'fecha_dato': 'measurement_date'}, inplace=True)

In [8]:
col_name = 'measurement_date'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            datetime64[ns]
------------------------------ 
np.na count:      0
------------------------------ 
np.na ratio:      0.0
------------------------------ 
distinct values:  18
------------------------------ 
top 10 values:
2016-05-28    931453
2016-06-28    929615
2016-04-28    928274
2016-03-28    925076
2016-02-28    920904
2016-01-28    916269
2015-12-28    912021
2015-11-28    906109
2015-10-28    892251
2015-09-28    865440
Name: measurement_date, dtype: int64
------------------------------ 
bottom 10 values:
2015-10-28    892251
2015-09-28    865440
2015-08-28    843201
2015-07-28    829817
2015-06-28    632110
2015-05-28    631957
2015-04-28    630367
2015-03-28    629209
2015-02-28    627394
2015-01-28    625457
Name: measurement_date, dtype: int64
------------------------------


## customer_id

In [9]:
d.rename(columns={'ncodpers': 'customer_id'}, inplace=True)

In [10]:
col_name = 'customer_id'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            int64
------------------------------ 
np.na count:      0
------------------------------ 
np.na ratio:      0.0
------------------------------ 
distinct values:  956645
------------------------------ 
top 10 values:
32752      18
1240915    18
1191755    18
1199943    18
1150783    18
1158971    18
1167159    18
1175347    18
1117999    18
1142563    18
Name: customer_id, dtype: int64
------------------------------ 
bottom 10 values:
350152     1
1514065    1
1462468    1
1531660    1
1452608    1
1244374    1
1542334    1
1513324    1
1501125    1
1500541    1
Name: customer_id, dtype: int64
------------------------------


## employment_index

In [11]:
d.rename(columns={'ind_empleado': 'employment_index'}, inplace=True)

In [12]:
col_name = 'employment_index'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  5
------------------------------ 
top 10 values:
N    14540073
B        3784
F        2675
A        2640
S          18
Name: employment_index, dtype: int64
------------------------------ 
bottom 10 values:
N    14540073
B        3784
F        2675
A        2640
S          18
Name: employment_index, dtype: int64
------------------------------


In [13]:
d.employment_index = d.employment_index.map({
        'N': 'not-employee',
        'B': 'ex-employee',
        'F': 'filial',
        'A': 'active',
        'S': 'passive'})

In [14]:
col_name = 'employment_index'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  5
------------------------------ 
top 10 values:
not-employee    14540073
ex-employee         3784
filial              2675
active              2640
passive               18
Name: employment_index, dtype: int64
------------------------------ 
bottom 10 values:
not-employee    14540073
ex-employee         3784
filial              2675
active              2640
passive               18
Name: employment_index, dtype: int64
------------------------------


## country_of_residence

In [15]:
d.rename(columns={'pais_residencia': 'country_of_residence'}, inplace=True)

In [16]:
col_name = 'country_of_residence'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  118
------------------------------ 
top 10 values:
ES    14479329
FR        5472
AR        5123
DE        4906
GB        4891
US        3882
CO        3737
IT        3126
RO        3106
MX        2729
Name: country_of_residence, dtype: int64
------------------------------ 
bottom 10 values:
LV    18
LY    18
IS    18
KH    18
KW    18
JM    13
DJ    12
ZW    12
BM     7
MT     3
Name: country_of_residence, dtype: int64
------------------------------


## gender

In [17]:
d.rename(columns={'sexo': 'gender'}, inplace=True)

In [18]:
col_name = 'gender'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27809
------------------------------ 
np.na ratio:      0.001907741304
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
V    7928767
H    6620348
Name: gender, dtype: int64
------------------------------ 
bottom 10 values:
V    7928767
H    6620348
Name: gender, dtype: int64
------------------------------


In [19]:
d.gender = d.gender.map({
        'V': 'male',
        'H': 'female'})

## age

In [20]:
col_name = 'age'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      0
------------------------------ 
np.na ratio:      0.0
------------------------------ 
distinct values:  122
------------------------------ 
top 10 values:
 23    829488
 24    785491
 22    783988
 21    722310
 25    513445
 20    444376
 26    374450
 43    345895
 44    344197
 42    340556
Name: age, dtype: int64
------------------------------ 
bottom 10 values:
113    127
115     82
116     70
114     28
117     15
127      9
163      8
164      4
126      3
118      1
Name: age, dtype: int64
------------------------------


In [21]:
def fix_age(x):
    try:
        a = int(x)
        if a < 0 or a > 110:
            return np.nan
        return a
    except:
        return np.nan

In [22]:
d.age = d.age.apply(fix_age)

In [23]:
col_name = 'age'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      28554
------------------------------ 
np.na ratio:      0.00195884948018
------------------------------ 
distinct values:  109
------------------------------ 
top 10 values:
23.0    829488
24.0    785491
22.0    783988
21.0    722310
25.0    513445
20.0    444376
26.0    374450
43.0    345895
44.0    344197
42.0    340556
Name: age, dtype: int64
------------------------------ 
bottom 10 values:
103.0    2154
3.0      1648
104.0    1470
105.0    1370
106.0     971
2.0       780
107.0     646
108.0     494
109.0     291
110.0     281
Name: age, dtype: int64
------------------------------


In [24]:
d.age.dropna().describe()

count    1.454837e+07
mean     4.018169e+01
std      1.717667e+01
min      2.000000e+00
25%      2.400000e+01
50%      3.900000e+01
75%      5.000000e+01
max      1.100000e+02
Name: age, dtype: float64

## first_contract_date

In [25]:
d.rename(columns={'fecha_alta': 'first_contract_date'}, inplace=True)

In [26]:
col_name = 'first_contract_date'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            datetime64[ns]
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  6756
------------------------------ 
top 10 values:
2014-07-28    60810
2014-10-03    57642
2014-08-04    48533
2013-10-14    43437
2013-08-03    35427
2012-08-06    27817
2014-07-21    27604
2012-08-03    26946
2012-08-02    26518
2013-10-11    26260
Name: first_contract_date, dtype: int64
------------------------------ 
bottom 10 values:
2014-06-28    12
2010-10-09    12
2013-11-01    12
2011-12-24    12
2012-08-26    11
2011-04-30    11
2015-05-31     9
2014-05-01     7
2009-12-25     5
2010-07-04     4
Name: first_contract_date, dtype: int64
------------------------------


## is_new_customer

In [27]:
d.rename(columns={'ind_nuevo': 'is_new_customer'}, inplace=True)

In [28]:
col_name = 'is_new_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
0.0    13712094
1.0      837096
Name: is_new_customer, dtype: int64
------------------------------ 
bottom 10 values:
0.0    13712094
1.0      837096
Name: is_new_customer, dtype: int64
------------------------------


In [29]:
d.is_new_customer = d.is_new_customer.map({
        0.0: 'no',
        1.0: 'yes'
    })

In [30]:
col_name = 'is_new_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
no     13712094
yes      837096
Name: is_new_customer, dtype: int64
------------------------------ 
bottom 10 values:
no     13712094
yes      837096
Name: is_new_customer, dtype: int64
------------------------------


## customer_for_months

In [31]:
d.rename(columns={'antiguedad': 'customer_for_months'}, inplace=True)

In [32]:
col_name = 'customer_for_months'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      0
------------------------------ 
np.na ratio:      0.0
------------------------------ 
distinct values:  260
------------------------------ 
top 10 values:
     12    246752
     21    232718
     10    216109
      9    200929
     23    193779
     33    187062
     45    182275
      8    182038
     24    172144
     44    169368
Name: customer_for_months, dtype: int64
------------------------------ 
bottom 10 values:
    249    2408
    250    1703
    251    1490
    252    1119
    253     660
    254     413
    255     263
    256     179
    257     102
-999999      41
Name: customer_for_months, dtype: int64
------------------------------


In [33]:
def fix_customer_for_months(x):
    try:
        a = int(x)
        if a < 0 or a > 1200:
            return np.nan
        return a
    except:
        return np.nan

In [34]:
d.customer_for_months = d.customer_for_months.apply(fix_customer_for_months)

In [35]:
col_name = 'customer_for_months'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      27775
------------------------------ 
np.na ratio:      0.00190540885032
------------------------------ 
distinct values:  258
------------------------------ 
top 10 values:
12.0    246752
21.0    232718
10.0    216109
9.0     200929
23.0    193779
33.0    187062
45.0    182275
8.0     182038
24.0    172144
44.0    169368
Name: customer_for_months, dtype: int64
------------------------------ 
bottom 10 values:
248.0    3382
249.0    2408
250.0    1703
251.0    1490
252.0    1119
253.0     660
254.0     413
255.0     263
256.0     179
257.0     102
Name: customer_for_months, dtype: int64
------------------------------


In [36]:
d.customer_for_months.dropna().describe()

count    1.454915e+07
mean     7.948280e+01
std      6.643134e+01
min      0.000000e+00
25%      2.300000e+01
50%      5.000000e+01
75%      1.350000e+02
max      2.570000e+02
Name: customer_for_months, dtype: float64

## is_primary_customer

In [37]:
d.rename(columns={'indrel': 'is_primary_customer'}, inplace=True)

In [38]:
col_name = 'is_primary_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
1.0     14522714
99.0       26476
Name: is_primary_customer, dtype: int64
------------------------------ 
bottom 10 values:
1.0     14522714
99.0       26476
Name: is_primary_customer, dtype: int64
------------------------------


In [39]:
d.is_primary_customer = d.is_primary_customer.map({
        1: 'yes',
        99: 'no'})

In [40]:
col_name = 'is_primary_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
yes    14522714
no        26476
Name: is_primary_customer, dtype: int64
------------------------------ 
bottom 10 values:
yes    14522714
no        26476
Name: is_primary_customer, dtype: int64
------------------------------


## last_primary_customer_date

In [41]:
d.rename(columns={'ult_fec_cli_1t': 'last_primary_customer_date'}, inplace=True)

In [42]:
col_name = 'last_primary_customer_date'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      14550448
------------------------------ 
np.na ratio:      0.998183704601
------------------------------ 
distinct values:  244
------------------------------ 
top 10 values:
2015-12-24    763
2015-12-28    521
2015-07-09    443
2015-07-06    405
2015-07-01    401
2015-07-21    391
2015-07-07    340
2015-07-17    330
2015-07-10    320
2015-07-28    311
Name: last_primary_customer_date, dtype: int64
------------------------------ 
bottom 10 values:
2015-08-14    51
2016-06-28    49
2016-03-24    49
2016-03-23    46
2015-08-19    44
2016-04-28    44
2015-08-06    41
2016-06-29    36
2016-01-04    34
2015-12-23    27
Name: last_primary_customer_date, dtype: int64
------------------------------


## customer_type

In [43]:
d.rename(columns={'indrel_1mes': 'customer_type'}, inplace=True)

In [44]:
col_name = 'customer_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      149804
------------------------------ 
np.na ratio:      0.010276790906
------------------------------ 
distinct values:  9
------------------------------ 
top 10 values:
1.0    9745138
1      4675108
3.0       2792
3         1585
P          874
2.0        740
2          577
4.0        223
4           83
Name: customer_type, dtype: int64
------------------------------ 
bottom 10 values:
1.0    9745138
1      4675108
3.0       2792
3         1585
P          874
2.0        740
2          577
4.0        223
4           83
Name: customer_type, dtype: int64
------------------------------


In [45]:
d.customer_type = d.customer_type.map({
        1: 'primary',
        1.0: 'primary',
        '1': 'primary',
        '1.0': 'primary',
        2: 'co-owner',
        2.0: 'co-owner',
        '2': 'co-owner',
        '2.0': 'co-owner',
        3: 'ex-primary',
        3.0: 'ex-primary',
        '3': 'ex-primary',
        '3.0': 'ex-primary',
        4: 'ex-co-owner',
        4.0: 'ex-co-owner',
        '4': 'ex-co-owner',
        '4.0': 'ex-co-owner',
        'P': 'potential'})

In [46]:
col_name = 'customer_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      149804
------------------------------ 
np.na ratio:      0.010276790906
------------------------------ 
distinct values:  5
------------------------------ 
top 10 values:
primary        14420246
ex-primary         4377
co-owner           1317
potential           874
ex-co-owner         306
Name: customer_type, dtype: int64
------------------------------ 
bottom 10 values:
primary        14420246
ex-primary         4377
co-owner           1317
potential           874
ex-co-owner         306
Name: customer_type, dtype: int64
------------------------------


## customer_relation_type

In [47]:
d.rename(columns={'tiprel_1mes': 'customer_relation_type'}, inplace=True)

In [48]:
col_name = 'customer_relation_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      149804
------------------------------ 
np.na ratio:      0.010276790906
------------------------------ 
distinct values:  5
------------------------------ 
top 10 values:
I    7840818
A    6580745
P       4683
R        870
N          4
Name: customer_relation_type, dtype: int64
------------------------------ 
bottom 10 values:
I    7840818
A    6580745
P       4683
R        870
N          4
Name: customer_relation_type, dtype: int64
------------------------------


In [49]:
d.customer_relation_type = d.customer_relation_type.map({
        'I': 'inactive',
        'A': 'active',
        'P': 'ex-customer',
        'R': 'potential'})

In [50]:
col_name = 'customer_relation_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      149808
------------------------------ 
np.na ratio:      0.0102770653123
------------------------------ 
distinct values:  4
------------------------------ 
top 10 values:
inactive       7840818
active         6580745
ex-customer       4683
potential          870
Name: customer_relation_type, dtype: int64
------------------------------ 
bottom 10 values:
inactive       7840818
active         6580745
ex-customer       4683
potential          870
Name: customer_relation_type, dtype: int64
------------------------------


## country_residence_bank_same

In [51]:
d.rename(columns={'indresi': 'country_residence_bank_same'}, inplace=True)

In [52]:
col_name = 'country_residence_bank_same'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
S    14479331
N       69859
Name: country_residence_bank_same, dtype: int64
------------------------------ 
bottom 10 values:
S    14479331
N       69859
Name: country_residence_bank_same, dtype: int64
------------------------------


In [53]:
d.country_residence_bank_same = d.country_residence_bank_same.map({
        'S': 'yes',
        'N': 'no'})

In [54]:
col_name = 'country_residence_bank_same'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
yes    14479331
no        69859
Name: country_residence_bank_same, dtype: int64
------------------------------ 
bottom 10 values:
yes    14479331
no        69859
Name: country_residence_bank_same, dtype: int64
------------------------------


## is_foreigner

In [55]:
d.rename(columns={'indext': 'is_foreigner'}, inplace=True)

In [56]:
col_name = 'is_foreigner'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
N    13858387
S      690803
Name: is_foreigner, dtype: int64
------------------------------ 
bottom 10 values:
N    13858387
S      690803
Name: is_foreigner, dtype: int64
------------------------------


In [57]:
d.is_foreigner = d.is_foreigner.map({
        'S': 'yes',
        'N': 'no'})

In [58]:
col_name = 'is_foreigner'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
no     13858387
yes      690803
Name: is_foreigner, dtype: int64
------------------------------ 
bottom 10 values:
no     13858387
yes      690803
Name: is_foreigner, dtype: int64
------------------------------


## is_spouse_of_an_employee

In [59]:
d.rename(columns={'conyuemp': 'is_spouse_of_an_employee'}, inplace=True)

In [60]:
col_name = 'is_spouse_of_an_employee'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      14575012
------------------------------ 
np.na ratio:      0.999868833781
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
N    1894
S      18
Name: is_spouse_of_an_employee, dtype: int64
------------------------------ 
bottom 10 values:
N    1894
S      18
Name: is_spouse_of_an_employee, dtype: int64
------------------------------


In [61]:
d.is_spouse_of_an_employee = d.is_spouse_of_an_employee.map({
        'S': 'yes',
        'N': 'no'})

In [62]:
col_name = 'is_spouse_of_an_employee'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      14575012
------------------------------ 
np.na ratio:      0.999868833781
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
no     1894
yes      18
Name: is_spouse_of_an_employee, dtype: int64
------------------------------ 
bottom 10 values:
no     1894
yes      18
Name: is_spouse_of_an_employee, dtype: int64
------------------------------


## join_channel

In [63]:
d.rename(columns={'canal_entrada': 'join_channel'}, inplace=True)

In [64]:
col_name = 'join_channel'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      188207
------------------------------ 
np.na ratio:      0.0129112973354
------------------------------ 
distinct values:  162
------------------------------ 
top 10 values:
KHE    4306935
KAT    3474042
KFC    3299057
KHQ     666008
KFA     435322
KHK     258002
KHM     217308
KHN     134739
KHD     124281
KAS      91805
Name: join_channel, dtype: int64
------------------------------ 
bottom 10 values:
KHA    54
KGU    30
KGC    30
KHS    27
KDB    18
KGN    18
KDI    18
KDL    12
025    12
KHR     2
Name: join_channel, dtype: int64
------------------------------


## is_dead

In [65]:
d.rename(columns={'indfall': 'is_dead'}, inplace=True)

In [66]:
col_name = 'is_dead'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
N    14512028
S       37162
Name: is_dead, dtype: int64
------------------------------ 
bottom 10 values:
N    14512028
S       37162
Name: is_dead, dtype: int64
------------------------------


In [67]:
d.is_dead = d.is_dead.map({
        'S': 'yes',
        'N': 'no'})

In [68]:
col_name = 'is_dead'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
no     14512028
yes       37162
Name: is_dead, dtype: int64
------------------------------ 
bottom 10 values:
no     14512028
yes       37162
Name: is_dead, dtype: int64
------------------------------


## address_type

In [69]:
d.rename(columns={'tipodom': 'address_type'}, inplace=True)

In [70]:
col_name = 'address_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      27735
------------------------------ 
np.na ratio:      0.00190266478717
------------------------------ 
distinct values:  1
------------------------------ 
top 10 values:
1.0    14549189
Name: address_type, dtype: int64
------------------------------ 
bottom 10 values:
1.0    14549189
Name: address_type, dtype: int64
------------------------------


In [71]:
d.address_type = d.address_type.map({
        1.0: 'address'})

In [72]:
col_name = 'address_type'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27735
------------------------------ 
np.na ratio:      0.00190266478717
------------------------------ 
distinct values:  1
------------------------------ 
top 10 values:
address    14549189
Name: address_type, dtype: int64
------------------------------ 
bottom 10 values:
address    14549189
Name: address_type, dtype: int64
------------------------------


## province_code

In [73]:
d.rename(columns={'cod_prov': 'province_code'}, inplace=True)

In [74]:
col_name = 'province_code'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      97587
------------------------------ 
np.na ratio:      0.00669462226736
------------------------------ 
distinct values:  52
------------------------------ 
top 10 values:
28.0    4707850
8.0     1363798
46.0     730300
41.0     645656
15.0     458037
30.0     424511
29.0     391569
50.0     365703
3.0      335544
11.0     314479
Name: province_code, dtype: int64
------------------------------ 
bottom 10 values:
49.0    54357
34.0    52593
40.0    45222
22.0    42970
5.0     41411
1.0     40295
44.0    24063
42.0    18921
52.0    10103
51.0     7731
Name: province_code, dtype: int64
------------------------------


In [75]:
def fix_province_code(c):
    try:
        return 'prov_' + str(int(c))
    except:
        return np.nan

In [76]:
d.province_code = d.province_code.apply(fix_province_code)

In [77]:
col_name = 'province_code'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      97587
------------------------------ 
np.na ratio:      0.00669462226736
------------------------------ 
distinct values:  52
------------------------------ 
top 10 values:
prov_28    4707850
prov_8     1363798
prov_46     730300
prov_41     645656
prov_15     458037
prov_30     424511
prov_29     391569
prov_50     365703
prov_3      335544
prov_11     314479
Name: province_code, dtype: int64
------------------------------ 
bottom 10 values:
prov_49    54357
prov_34    52593
prov_40    45222
prov_22    42970
prov_5     41411
prov_1     40295
prov_44    24063
prov_42    18921
prov_52    10103
prov_51     7731
Name: province_code, dtype: int64
------------------------------


## province_name

In [78]:
d.rename(columns={'nomprov': 'province_name'}, inplace=True)

In [79]:
col_name = 'province_name'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      97587
------------------------------ 
np.na ratio:      0.00669462226736
------------------------------ 
distinct values:  52
------------------------------ 
top 10 values:
MADRID       4707850
BARCELONA    1363798
VALENCIA      730300
SEVILLA       645656
CORUÑA, A     458037
MURCIA        424511
MALAGA        391569
ZARAGOZA      365703
ALICANTE      335544
CADIZ         314479
Name: province_name, dtype: int64
------------------------------ 
bottom 10 values:
ZAMORA      54357
PALENCIA    52593
SEGOVIA     45222
HUESCA      42970
AVILA       41411
ALAVA       40295
TERUEL      24063
SORIA       18921
MELILLA     10103
CEUTA        7731
Name: province_name, dtype: int64
------------------------------


In [80]:
del d['province_name']

## is_active_customer

In [81]:
d.rename(columns={'ind_actividad_cliente': 'is_active_customer'}, inplace=True)

In [82]:
col_name = 'is_active_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
0.0    7918666
1.0    6630524
Name: is_active_customer, dtype: int64
------------------------------ 
bottom 10 values:
0.0    7918666
1.0    6630524
Name: is_active_customer, dtype: int64
------------------------------


In [83]:
d.is_active_customer = d.is_active_customer.map({
        1.0: 'yes',
        0.0: 'no'})

In [84]:
col_name = 'is_active_customer'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      27734
------------------------------ 
np.na ratio:      0.00190259618559
------------------------------ 
distinct values:  2
------------------------------ 
top 10 values:
no     7918666
yes    6630524
Name: is_active_customer, dtype: int64
------------------------------ 
bottom 10 values:
no     7918666
yes    6630524
Name: is_active_customer, dtype: int64
------------------------------


## household_income

In [85]:
d.rename(columns={'renta': 'household_income'}, inplace=True)

In [86]:
col_name = 'household_income'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      2794375
------------------------------ 
np.na ratio:      0.191698536673
------------------------------ 
distinct values:  1037397
------------------------------ 
top 10 values:
         NA    227965
451931.22        5582
463625.16        1743
181042.2         1493
128318.52        1404
105260.88        1354
488798.49        1332
127141.5         1183
283325.67        1148
132335.73        1047
Name: household_income, dtype: int64
------------------------------ 
bottom 10 values:
  117387.66    1
   91993.32    1
  143050.95    1
  151665.90    1
  258590.94    1
   85901.37    1
   52709.49    1
   46756.56    1
   66247.35    1
   84947.31    1
Name: household_income, dtype: int64
------------------------------


In [87]:
def fix_household_income(i):
    try:
        return float(i)
    except:
        return np.nan

In [88]:
d.household_income = d.household_income.apply(fix_household_income)

In [89]:
col_name = 'household_income'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            float64
------------------------------ 
np.na count:      3022340
------------------------------ 
np.na ratio:      0.207337295578
------------------------------ 
distinct values:  592576
------------------------------ 
top 10 values:
451931.22    5936
463625.16    1854
181042.20    1584
128318.52    1495
105260.88    1444
488798.49    1416
127141.50    1256
283325.67    1218
132335.73    1114
236690.34    1076
Name: household_income, dtype: int64
------------------------------ 
bottom 10 values:
85126.95     1
53730.57     1
52828.74     1
88762.29     1
117280.89    1
68446.71     1
79364.79     1
130394.61    1
84029.04     1
76863.54     1
Name: household_income, dtype: int64
------------------------------


In [90]:
d.household_income.dropna().describe()

count    1.155458e+07
mean     1.342442e+05
std      2.307233e+05
min      1.202730e+03
25%      6.869931e+04
50%      1.018289e+05
75%      1.559304e+05
max      2.889440e+07
Name: household_income, dtype: float64

## customer_segment

In [91]:
d.rename(columns={'segmento': 'customer_segment'}, inplace=True)

In [92]:
col_name = 'customer_segment'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      191616
------------------------------ 
np.na ratio:      0.0131451601175
------------------------------ 
distinct values:  3
------------------------------ 
top 10 values:
02 - PARTICULARES     8505598
03 - UNIVERSITARIO    5281607
01 - TOP               598103
Name: customer_segment, dtype: int64
------------------------------ 
bottom 10 values:
02 - PARTICULARES     8505598
03 - UNIVERSITARIO    5281607
01 - TOP               598103
Name: customer_segment, dtype: int64
------------------------------


In [93]:
d.customer_segment = d.customer_segment.map({
        '02 - PARTICULARES': 'individual',
        '01 - TOP': 'vip',
        '03 - UNIVERSITARIO': 'college-graduated'})

In [94]:
col_name = 'customer_segment'

print '-' * 30, '\n', 'dtype:           ', d[col_name].dtype
print '-' * 30, '\n', 'np.na count:     ', d[col_name].isnull().sum()
print '-' * 30, '\n', 'np.na ratio:     ', 1.0 * d[col_name].isnull().sum() / len(d)
print '-' * 30, '\n', 'distinct values: ', len(d[col_name].value_counts())
print '-' * 30, '\n', 'top 10 values:'
print d[col_name].value_counts().head(10)
print '-' * 30, '\n', 'bottom 10 values:'
print d[col_name].value_counts().tail(10)
print '-' * 30

------------------------------ 
dtype:            object
------------------------------ 
np.na count:      191616
------------------------------ 
np.na ratio:      0.0131451601175
------------------------------ 
distinct values:  3
------------------------------ 
top 10 values:
individual           8505598
college-graduated    5281607
vip                   598103
Name: customer_segment, dtype: int64
------------------------------ 
bottom 10 values:
individual           8505598
college-graduated    5281607
vip                   598103
Name: customer_segment, dtype: int64
------------------------------


In [95]:
d.head(10).T

,0,1,2,3,4,5,6,7,8,9
measurement_date,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00,2015-01-28 00:00:00
customer_id,1375586,1050611,1050612,1050613,1050614,1050615,1050616,1050617,1050619,1050620
employment_index,not-employee,not-employee,not-employee,not-employee,not-employee,not-employee,not-employee,not-employee,not-employee,not-employee
country_of_residence,ES,ES,ES,ES,ES,ES,ES,ES,ES,ES
gender,female,male,male,female,male,female,female,female,female,female
age,35,23,23,22,23,23,23,23,24,23
first_contract_date,2015-01-12 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00,2012-08-10 00:00:00
is_new_customer,no,no,no,no,no,no,no,no,no,no
customer_for_months,6,35,35,35,35,35,35,35,35,35
is_primary_customer,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes


## Write the Data to a File

In [96]:
d.to_csv('tmp/clean.csv', index=False)